In [ ]:
import sys
sys.executable

In [ ]:
%load_ext autoreload
%autoreload 2
from micron2.data import pull_tiles
from micron2.data import load_as_anndata

import numpy as np
import pandas as pd
import pytiff
import h5py
import cv2
import os

datahome = '/home/ingn/tmp/micron2-data/rawdata'

cells = pd.read_csv(f'{datahome}/201021_BreastFFPE_Final/201021_BreastFFPE_Final_3_cells.csv', 
                    index_col=0, header=0)
nuclei_img = f'{datahome}/201021_BreastFFPE_Final/201021_BreastFFPE_Final_2_nuclei.tif'

imagefs = !ls /home/ingn/tmp/micron2-data/rawdata/201021_BreastFFPE_Final/images/*.tif
dapi_images = [f for f in imagefs if 'DAPI' in f]
non_dapi_images = [f for f in imagefs if 'DAPI' not in f]
non_dapi_images = [f for f in non_dapi_images if 'Blank' not in f]
non_dapi_images = [f for f in non_dapi_images if 'Empty' not in f]

channel_names = [os.path.basename(x) for x in non_dapi_images]
channel_names = [x.replace(f'.tif','') for x in channel_names]
channel_names = [x.split('_')[-2] for x in channel_names]
channel_names = ["DAPI"] + channel_names
print(len(channel_names))

image_paths = [dapi_images[0]] + non_dapi_images
print(len(image_paths))

pull_tiles(image_paths, 
           out_file='/home/ingn/tmp/micron2-data/dataset_tiles.hdf5', 
           #nuclei_img=nuclei_img,
           size=128,
           scale_factor=1,
           overlap=0.5,
           channel_names=channel_names,
           debug=False
           )

In [ ]:
!ls -lha *.hdf5

In [ ]:
adata = load_as_anndata('/home/ingn/tmp/micron2-data/dataset_tiles.hdf5', obs_names='meta/Tile_IDs',
                        obsm='meta/bounding_boxes')
adata

In [ ]:
import scanpy as sc
sc.pl.embedding(adata, basis='coordinates', s=100)